# Import of packages

In [1]:
'''To import the required packages.'''
import pandas as pd
import numpy as np
import networkx as nx
import collections
import matplotlib.pyplot as plt
import math
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from kneed import KneeLocator
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA

#import osmnx as ox

# Settings

In [2]:
'''To display all output results of a Jupyter cell.'''
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
'''To ensure that the output results of extensive output results are not truncated.'''
#pd.options.display.max_rows = 4000

'To ensure that the output results of extensive output results are not truncated.'

In [4]:
'''To change the width of the Notebook to see the output on the screen'''
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

'To change the width of the Notebook to see the output on the screen'

# **Dutch railway system**

# Import of the Dutch railway datasets

In [5]:
'''To register the GitHub link with the Dutch data as a variable.'''
datalink = "https://raw.githubusercontent.com/polkuleuven/Thesis_Train/main/gtfs_train_Netherlands_1503/"

'To register the GitHub link with the Dutch data as a variable.'

In [6]:
'''Import all the GTFS data'''
#To import the agency dataset that contains limited information about the Dutch NS railway agency.
agency_Netherlands = pd.read_csv(datalink + "agency.txt", sep=",")
#To import the stops dataset that contains information about the ids, the names and the geographical coordinates of the Dutch NS railway stations.
stops_Netherlands = pd.read_csv(datalink + "stops.txt", sep=",")
#To import the feed_info dataset that contains limited information about the Dutch NS railway feed.
feed_info_Netherlands = pd.read_csv(datalink + "feed_info.txt", sep=",")
#To import the transfers dataset that gives the minimum transfer time to switch routes at each Belgian railway station.
transfers_not_cleaned_Netherlands = pd.read_csv(datalink + "transfers.txt", sep=",")
#To import the routes dataset that provides the id, the name and the type of vehicle used for all Dutch NS railway routes.
routes_Netherlands = pd.read_csv(datalink + "routes.txt", sep=",")
#To import the trips dataset that gives for all routes an overview of the trips and the headsigns of these trips belonging to the Dutch NS railway route.
#The service_id is an indication of all the dates this trip is valid (consultable in the calendar_dates dataset).
trips_Netherlands = pd.read_csv(datalink + "trips.txt", sep=",")
#To import the calendar_dates dataset that gives for each service_id all the exact dates when that service_id is valid.
calendar_dates_Netherlands = pd.read_csv(datalink + "calendar_dates.txt", sep=",")

'Import all the GTFS data'

In [7]:
#stop_times_Netherlands = pd.read_csv("stop_times_Netherlands.csv", sep=",")

In [8]:
#To import the stop_times dataset that gives for all trips an overview of the ids of the stations served and the sequence in which these stations are served. 
#In addition, for all the trips the arrival and departure times at the stations served are given.
stop_times_range = [*range(2, 19)]
stop_times_Netherlands = pd.read_csv(datalink + "stop_times-1.csv", sep=",")
for index in stop_times_range:
    stop_times_Netherlands = pd.concat([stop_times_Netherlands, pd.read_csv(datalink + "stop_times-" + str(index)+ ".csv", sep=",")])
stop_times_Netherlands

,trip_id,stop_sequence,stop_id,stop_headsign,arrival_time,departure_time,pickup_type,drop_off_type,timepoint,shape_dist_traveled,fare_units_traveled
0,127986896,24,15250,NaN,13:36:00,13:36:00,1,0,0.0,12272.0,12272.0
1,127986896,11,15131,NaN,13:15:24,13:15:42,0,0,0.0,5407.0,5407.0
2,127986896,8,15661,NaN,13:09:12,13:09:30,0,0,0.0,3632.0,3632.0
3,127986896,6,14784,NaN,13:05:36,13:05:54,0,0,0.0,2855.0,2855.0
4,127986896,4,15774,NaN,13:00:38,13:00:56,0,0,0.0,1704.0,1704.0
...,...,...,...,...,...,...,...,...,...,...,...
507727,128464528,16,537637,NaN,16:02:00,16:02:00,0,0,0.0,9600.0,9903.0
507728,128464529,14,1420429,Veldhoven Sondervick,15:28:00,15:28:00,0,0,0.0,7745.0,8044.0
507729,128464529,3,939748,NaN,15:12:00,15:12:00,0,0,0.0,1146.0,1168.0
507730,128464537,14,1420429,Veldhoven Sondervick,17:28:00,17:28:00,0,0,0.0,7745.0,8044.0


# Cleaning of the Dutch railway data

In [9]:
'''To clean the routes_Netherlands df'''
#To keep the train routes
routes_cleaned_Netherlands = routes_Netherlands[routes_Netherlands['route_type'] == 2]
routes_cleaned_Netherlands = routes_cleaned_Netherlands.astype(str)
routes_cleaned_Netherlands.describe(include=['object'])

#To change the route_id object datatype to a NumPy int64 datatype
routes_cleaned_Netherlands.loc[:,'route_id'] = routes_cleaned_Netherlands.loc[:,'route_id'].astype(np.int64)

'To clean the routes_Netherlands df'

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_text_color,route_url
count,145,145,145,145,145,145,145,145,145
unique,145,11,15,144,1,1,1,1,1
top,17782,IFF:NS,Sprinter,Nachtnettrein Utrecht Centraal <-> Rotterdam C...,nan,2,nan,nan,nan
freq,1,87,47,2,145,145,145,145,145


In [10]:
'''To clean the calendar_dates_Netherlands df '''
#To filter the dates from the selected begin to the end date
begin_date = 20210314
end_date = 20210713
calendar_dates_cleaned_Netherlands = calendar_dates_Netherlands.copy()
calendar_dates_cleaned_Netherlands = calendar_dates_cleaned_Netherlands.drop(calendar_dates_cleaned_Netherlands[(calendar_dates_cleaned_Netherlands['date'] > end_date) | (calendar_dates_cleaned_Netherlands['date'] < begin_date)].index)
calendar_dates_cleaned_Netherlands

'To clean the calendar_dates_Netherlands df '

,service_id,date,exception_type
0,1,20210314,1
1,2,20210314,1
2,2,20210412,1
3,2,20210419,1
4,2,20210426,1
...,...,...,...
181776,4074,20210713,1
181779,4075,20210712,1
181780,4075,20210713,1
181784,4076,20210712,1


In [11]:
# To define a definition to remove the accents from a string
def remove_accents(text):
    import unicodedata
    try:
        text = unicode(text, 'utf-8')
    except NameError:
        pass
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
    return str(text)

In [12]:
''' To clean the stops df '''
#To take from the stops_initial_Netherlands df all stop_ids that contain a 'stoparea:' to get the correct stop coordinates
stops_cleaned_Netherlands = stops_Netherlands[stops_Netherlands['stop_id'].str.contains('stoparea:')]

#To remove the accents from the accented characters and to convert the remaining characters to uppercase characters
stops_cleaned_Netherlands.loc[:,'stop_name'] = stops_cleaned_Netherlands.loc[:,'stop_name'].apply(remove_accents)
stops_cleaned_Netherlands.loc[:,'stop_name'] = stops_cleaned_Netherlands.loc[:,'stop_name'].str.upper()
stops_cleaned_Netherlands

#To initialize the Nominatim API to get the location from the input string 
geolocator = Nominatim(user_agent="application")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=0.2)

#To get the location with the geolocator.reverse() function and to extract the country from the location instance
country_list = []
i = 0
for index, row in stops_cleaned_Netherlands.iterrows():
    i += 1
    print(i)
    latitude = row['stop_lat']
    longitude = row['stop_lon']
    # To assign the latitude and longitude into a geolocator.reverse() method
    location = reverse((latitude, longitude), language='en', exactly_one=True)
    # To get the country from the given list and parsed into a dictionary with raw function()
    address = location.raw['address']
    country = address.get('country', '')
    country_list.append(country)

#To add the values of country_list as a new attribute country 
stops_cleaned_Netherlands.loc[:,'country'] = country_list
stops_cleaned_Netherlands
stops_cleaned_Netherlands.to_csv('stops_cleaned_Netherlands_country.csv')

' To clean the stops df '

/Users/pol/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,stop_id,stop_code,stop_name,stop_lat,stop_lon,location_type,parent_station,stop_timezone,wheelchair_boarding,platform_code,zone_id
46635,stoparea:40185,dresdn,DRESDEN-NEUSTADT,51.065933,13.740603,1,NaN,Europe/Amsterdam,0.0,NaN,NaN
46636,stoparea:40186,usti,USTI NAD LABEM,50.658207,14.029750,1,NaN,Europe/Amsterdam,0.0,NaN,NaN
46637,stoparea:37036,lillee,LILLE EUROPE,50.639444,3.075000,1,NaN,Europe/Amsterdam,0.0,NaN,NaN
46638,stoparea:40188,roskil,ROSKILDE,55.638584,12.088194,1,NaN,Europe/Amsterdam,0.0,NaN,NaN
46639,stoparea:40189,decin,DECIN HL.N.,50.773490,14.200820,1,NaN,Europe/Amsterdam,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
61569,stoparea:195075,NaN,"WEERT, BASSIN",51.257289,5.706368,1,NaN,NaN,0.0,NaN,NaN
61570,stoparea:32953,NaN,"AMSTERDAM, ARTIS",52.366650,4.911110,1,NaN,NaN,0.0,NaN,NaN
61571,stoparea:2612,NaN,"AMSTERDAM, LUTMASTRAAT",52.351663,4.903220,1,NaN,NaN,0.0,NaN,NaN
61572,stoparea:1133,NaN,"IJLST, STATION",53.014301,5.615621,1,NaN,NaN,0.0,NaN,NaN


1


RateLimiter caught an error, retrying (0/2 tries). Called with (*((51.0659329, 13.740602599999999),), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/pol/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto


...

RateLimiter caught an error, retrying (0/2 tries). Called with (*((52.11309, 5.086825),), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/pol/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto
    return s

426


RateLimiter caught an error, retrying (0/2 tries). Called with (*((52.105799, 5.0930919999999995),), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/pol/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto
 

RateLimiter swallowed an error after 2 retries. Called with (*((52.105799, 5.0930919999999995),), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/pol/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/Users/pol/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/pol/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto
    

AttributeError: 'NoneType' object has no attribute 'raw'

In [13]:
'''To clean the stop_times df'''
stop_times_cleaned_Netherlands = stop_times_Netherlands.copy()
stop_times_cleaned_Netherlands.loc[:,'stop_id'] = stop_times_cleaned_Netherlands.stop_id.apply(str)
stop_times_cleaned_Netherlands = pd.merge(stop_times_cleaned_Netherlands, stops_Netherlands[['stop_id', 'stop_name']], on='stop_id')
stop_times_cleaned_Netherlands.loc[:,'stop_name'] = stop_times_cleaned_Netherlands.loc[:,'stop_name'].apply(remove_accents)
stop_times_cleaned_Netherlands.loc[:,'stop_name'] = stop_times_cleaned_Netherlands.loc[:,'stop_name'].str.upper()
stop_times_cleaned_Netherlands

'To clean the stop_times df'

,trip_id,stop_sequence,stop_id,stop_headsign,arrival_time,departure_time,pickup_type,drop_off_type,timepoint,shape_dist_traveled,fare_units_traveled,stop_name
0,127986896,24,15250,NaN,13:36:00,13:36:00,1,0,0.0,12272.0,12272.0,"AMSTERDAM, MATTERHORN"
1,127986859,24,15250,NaN,10:01:00,10:01:00,1,0,0.0,12272.0,12272.0,"AMSTERDAM, MATTERHORN"
2,127986890,24,15250,NaN,13:06:00,13:06:00,1,0,0.0,12272.0,12272.0,"AMSTERDAM, MATTERHORN"
3,127986998,24,15250,NaN,22:05:00,22:05:00,1,0,0.0,12272.0,12272.0,"AMSTERDAM, MATTERHORN"
4,127986861,24,15250,NaN,10:32:00,10:32:00,1,0,0.0,12272.0,12272.0,"AMSTERDAM, MATTERHORN"
...,...,...,...,...,...,...,...,...,...,...,...,...
10945566,130566937,28,2383583,NaN,09:32:00,09:32:00,0,0,0.0,NaN,16928.0,"HEES, HEESSTRAAT"
10945567,130566801,26,2383715,NaN,09:31:00,09:31:00,0,0,0.0,NaN,16085.0,"HEES, TOMBESTRAAT"
10945568,130566937,26,2383715,NaN,09:31:00,09:31:00,0,0,0.0,NaN,16085.0,"HEES, TOMBESTRAAT"
10945569,130566801,3,2383782,NaN,08:56:00,08:56:00,0,0,0.0,NaN,972.0,"SCHOONBEEK, APPELVELDSTRAAT"


In [ ]:
stops_cleaned_Netherlands = pd.read_csv("stops_cleaned_Netherlands_country.csv", sep=",")

### To merge the files

In [14]:
'''To select all required fields'''
agency_cleaned_Netherlands = agency_Netherlands[['agency_id', 'agency_name', 'agency_url', 'agency_timezone']]
routes_cleaned_Netherlands = routes_cleaned_Netherlands[['route_id', 'agency_id', 'route_short_name', 'route_long_name', 'route_type']]
trips_cleaned_Netherlands = trips_Netherlands[['trip_id', 'route_id', 'service_id', 'trip_headsign']]
calendar_dates_cleaned_Netherlands = calendar_dates_cleaned_Netherlands[['service_id', 'date']]
stops_cleaned_Netherlands = stops_cleaned_Netherlands[['stop_name', 'stop_lat', 'stop_lon']]
stop_times_cleaned_Netherlands = stop_times_cleaned_Netherlands[['trip_id', 'stop_name', 'arrival_time', 'departure_time', 'stop_sequence']]

'To select all required fields'

In [15]:
''' To merge the Dutch files '''
#To merge the stop_times df with the stops df on stop_id
stop_times_stops_Netherlands = pd.merge(stop_times_cleaned_Netherlands, stops_cleaned_Netherlands[['stop_name', 'stop_lat', 'stop_lon']], on='stop_name')

#To merge the trips df with the routes df on route_id
routes_trips_Netherlands = pd.merge(routes_cleaned_Netherlands[['route_id']], trips_cleaned_Netherlands, on='route_id')

#To merge the stop_times_stops df with the trips_routes df on trip_id
routes_trips_stop_times_stops_Netherlands = pd.merge(routes_trips_Netherlands, stop_times_stops_Netherlands, on='trip_id')

#To take only the service_ids present in both the routes_trips_stop_times_stops df and the calendar_dates df into account
calendar_dates_cleaned_unique_Netherlands = calendar_dates_cleaned_Netherlands['service_id'].unique()
routes_trips_stop_times_stops_calendar_dates_Netherlands = routes_trips_stop_times_stops_Netherlands[(routes_trips_stop_times_stops_Netherlands['service_id'].isin(calendar_dates_cleaned_unique_Netherlands))]

' To merge the Dutch files '

In [16]:
routes_trips_stop_times_stops_calendar_dates_Netherlands

,route_id,trip_id,service_id,trip_headsign,stop_name,arrival_time,departure_time,stop_sequence,stop_lat,stop_lon
0,67394,121351438,1810,Eindhoven Centraal,DEN HAAG CENTRAAL,20:47:00,20:47:00,1,52.081131,4.324054
1,67394,121351438,1810,Eindhoven Centraal,TILBURG,21:51:00,21:53:00,18,51.560548,5.083457
2,67394,121351438,1810,Eindhoven Centraal,EINDHOVEN CENTRAAL,22:15:00,22:15:00,23,51.442376,5.479941
3,67394,121351438,1810,Eindhoven Centraal,ROTTERDAM CENTRAAL,21:12:00,21:14:00,8,51.924383,4.469746
4,67394,121351438,1810,Eindhoven Centraal,DELFT,21:00:00,21:00:00,5,52.006539,4.356516
...,...,...,...,...,...,...,...,...,...,...
341374,20895,123745346,1868,Leiden Centraal,AMSTERDAM BIJLMER ARENA,26:39:00,26:39:00,1,52.312204,4.947109
341375,20895,123745346,1868,Leiden Centraal,SCHIPHOL AIRPORT,26:50:00,27:03:00,4,52.309456,4.762284
341376,20895,123745344,1868,Amsterdam Bijlmer ArenA,LEIDEN CENTRAAL,25:47:00,25:47:00,1,52.166353,4.482068
341377,20895,123745344,1868,Amsterdam Bijlmer ArenA,AMSTERDAM BIJLMER ARENA,26:22:00,26:22:00,8,52.312204,4.947109


# Preparation space-of-stops

In [ ]:
# Verderwerken met routes_trips_stop_times_stops_calendar_dates_Netherlands of deze dataframe hernoemd naar een gemakkelijkere naam